In [4]:
import pandas as pd
import numpy as np
import collections

path = "/Users/yanbingliu/Dropbox/infocom23-dataset/interm_processed"
data_path = "/Users/yanbingliu/Dropbox/infocom23-dataset/grid"
interm_path = "/Users/yanbingliu/Dropbox/infocom23-dataset/interm_result"
path_bw = "/Users/yanbingliu/Dropbox/infocom23-dataset/bandwidth"

grid_size = 0.0005
sample_limit = 0
time_limit = 0

operator_list = ["A", "V", "T"]
region_list=["A-R1", "A-R2", "A-R3", "A-R4", "A-R5", "V-R6", "V-R7", "V-R8", "V-R9", "V-R10", "V-R11", "T-R12", "T-R13"]

#ue_type_list = ["Pixel5", "Pixel4a"]
ue_type_list = ["Pixel5"]
#ue_type_list = ["Pixel4a"]
target_grid_type_list = ["all"]
rat_list = ["5gm", "5gs"]

dataset_stat = []

region_res = {}

rsrp_threshold = -110

thput_sample_thres = 10
dominate_thres = 0.01

thput_factor_list = [0.1,0.2,0.3,0.4,0.5, 0.6, 0.7, 0.8, 0.9]

omega_list = [0.1, 0.3, 0.5, 0.7, 0.9]
gamma_list = [10, 20, 30, 40, 50]

freq_to_bandwidth = {675:15, 850:20, 1025:5, 5110:10, 5330:10, 9820:10, 9840:5, 
52740:20, 52940:20, 52941:20, 53139:20, 53140:20, 53141:20, 53339:20, 53341:20, 53539:20,
66486:10, 66661:5, 66686:10, 66936:10, 67086:10}

distribution = {}

for operator in operator_list:
    for ue_type in ue_type_list:
        mccmnc = ""
        if operator.find("A") >= 0:
            mccmnc = "310410"
        elif operator.find("V") >= 0:
            mccmnc = "311480"
        elif operator.find("T") >= 0:
            mccmnc = "310260"

        bw_freq = {}
        lte_bw_freq = {}
            
        total_instance_num = 0
        
        cell_bw = {}
        
        rsrp_grid = {}
        
        with open(path_bw + "/" + operator + "_bw_channel_list.csv", 'r', encoding='utf-8-sig') as lines:
            for line in lines:
                if line.startswith("channel"):
                    continue
                items = line.strip().split(',')

                freq = int(items[0])
                bw = items[1]
                if '/' in bw:
                    bw = bw.split('/')[0]
                
                bw_freq[freq] = int(bw)

        for region in region_list:
            if region.find(operator) < 0:
                continue
            
            distribution[region] = []
            
            line_cell = []
            
            cell_num_grid = {}
            nr_cell_gps = {}
            max_bw_grid = {}
            cell_bw_grid = {}
            cell_set_bw_grid = {}
            
            with open(data_path + "/" + region + "_rss_cell_20210405_20211226_" + mccmnc + "_"+ str(grid_size) + "_0_" + str(time_limit) + ".csv", 'r', encoding='utf-8-sig') as lines:
                for line in lines:
                    if line.startswith("grid"):
                        line_cell.append(line.strip('\n') + ',bandwidth')
                        continue
                    items = line.strip().split(',')
                    lat = float(items[0])
                    lon = float(items[1])
                    gps_str = str(items[0]) + str(items[1])

                    if items[13] == 'None':
                        continue
                    rsrp_med = float(items[13])
                    rsrp_0 = float(items[10])
                    rsrp_10 = float(items[11])
                    rsrp_25 = float(items[12])
                    rsrp_50 = float(items[13])
                    rsrp_75 = float(items[14])
                    rsrp_90 = float(items[15])
                    rsrp_100 = float(items[16])

                    cell_id = items[3]
                    freq = int(items[2])
                    cell_str = items[3] + '-' + items[2]
                    
                    if gps_str not in cell_bw_grid:
                        cell_bw_grid[gps_str] = {}
                        
                    if gps_str not in rsrp_grid:
                        rsrp_grid[gps_str] = [0, 0, 0, []]
                    
                    bw = 0
                    if freq in bw_freq:
                        bw = bw_freq[freq]
                    elif freq > 2000000:
                        bw = 100
                    
                    if rsrp_50 > -110:
                        cell_bw_grid[gps_str][cell_str] = [lat, lon, freq, bw]
                        cell_bw[cell_str] = [freq, bw]
                        line_cell.append(line.strip('\n') + ',' + str(bw))
                        
                        if bw >= 50:
                        
                            rsrp_grid[gps_str][3].append(rsrp_50)
                            sorted(rsrp_grid[gps_str][3], reverse=True)

                            rsrp_grid[gps_str][0] = len(rsrp_grid[gps_str][3])
                            rsrp_grid[gps_str][1] = rsrp_grid[gps_str][3][0]
                            rsrp_grid[gps_str][2] = rsrp_grid[gps_str][3][3] if rsrp_grid[gps_str][0] >= 4 else 0

                            #print(rsrp_grid[gps_str][1])
                            #print(rsrp_grid[gps_str][2])
            
            line_cell_set = []
            cell_set_grid_stat = {}
            
            with open(data_path + "/" + region + "_grid_cell_set_pixel5_20210405_20220127_" + mccmnc + "_"+ str(grid_size) + "_0_" + str(time_limit) + "_v3.csv", 'r', encoding='utf-8-sig') as lines:
                for line in lines:
                    if line.startswith("grid"):
                        line_tmp = "grid_lat,grid_lon,pcell_cid,pcell_freq,nrcell_cid,nrcell_freq,scell1_cid,scell1_freq,scell2_cid,scell2_freq,scell3_cid,scell3_freq,nrscell1_cid,nrscell1_freq,nrscell2_cid,nrscell2_freq,nrscell3_cid,nrscell3_freq,cell_set_type,bandwidth,total_avail_sample,avail_sample,avail_sample_ratio,thput_sample_num,thput_sample,thput_sample_ratio,thput_avg,thput_std,thput_0,thput_10,thput_25,thput_50,thput_75,thput_90,thput_100"
                        line_cell_set.append(line_tmp)
                        continue
                    items = line.strip().split(',')
                    lat = float(items[1])
                    lon = float(items[0])
                    gps_str = str(items[1]) + str(items[0])
                    
                    total_avail_sample = int(items[19])
                    if total_avail_sample < 100:
                        continue
                    
                    if gps_str not in cell_bw_grid:
                        cell_bw_grid[gps_str] = {}
                        
                    if gps_str not in cell_set_bw_grid:
                        cell_set_bw_grid[gps_str] = {}
                        
                    if gps_str not in cell_set_grid_stat:
                        cell_set_grid_stat[gps_str] = [str(items[1]), str(items[0]), 0, 0, 0]
                        
                    cell_set_str = items[2] + '-' + items[3] + items[4] + '-' + items[5] + items[6] + '-' + items[7] + items[8] + '-' + items[9] + items[10] + '-' + items[11] + items[12] + '-' + items[13] + items[14] + '-' + items[15] + items[16] + '-' + items[17]
                    
                    total_bw = 0
                    ratio = float(items[21])
                    
                    if items[3] != 'None':
                        freq = int(items[3])
                        cell_str = items[2] + '-' + items[3]
                        bw = 0
                        if freq in bw_freq:
                            bw = bw_freq[freq]
                        elif freq > 2000000:
                            bw = 100
                        cell_bw_grid[gps_str][cell_str] = [lat, lon, freq, bw]
                        cell_bw[cell_str] = [freq, bw]
                        total_bw += bw
                    if items[7] != 'None':
                        freq = int(items[7])
                        cell_str = items[6] + '-' + items[7]
                        bw = 0
                        if freq in bw_freq:
                            bw = bw_freq[freq]
                        elif freq > 2000000:
                            bw = 100
                        cell_bw_grid[gps_str][cell_str] = [lat, lon, freq, bw]
                        cell_bw[cell_str] = [freq, bw]
                        total_bw += bw
                    if items[9] != 'None':
                        freq = int(items[9])
                        cell_str = items[8] + '-' + items[9]
                        bw = 0
                        if freq in bw_freq:
                            bw = bw_freq[freq]
                        elif freq > 2000000:
                            bw = 100
                        cell_bw_grid[gps_str][cell_str] = [lat, lon, freq, bw]
                        cell_bw[cell_str] = [freq, bw]
                        total_bw += bw
                    if items[11] != 'None':
                        freq = int(items[11])
                        cell_str = items[10] + '-' + items[11]
                        bw = 0
                        if freq in bw_freq:
                            bw = bw_freq[freq]
                        elif freq > 2000000:
                            bw = 100
                        cell_bw_grid[gps_str][cell_str] = [lat, lon, freq, bw]
                        cell_bw[cell_str] = [freq, bw]
                        total_bw += bw
                    if items[5] != 'None':
                        freq = int(items[5])
                        cell_str = items[4] + '-' + items[5]
                        bw = 0
                        if freq in bw_freq:
                            bw = bw_freq[freq]
                        elif freq > 2000000:
                            bw = 100
                        cell_bw_grid[gps_str][cell_str] = [lat, lon, freq, bw]
                        cell_bw[cell_str] = [freq, bw]
                        total_bw += bw
                    if items[13] != 'None':
                        freq = int(items[13])
                        cell_str = items[12] + '-' + items[13]
                        bw = 0
                        if freq in bw_freq:
                            bw = bw_freq[freq]
                        elif freq > 2000000:
                            bw = 100
                        cell_bw_grid[gps_str][cell_str] = [lat, lon, freq, bw]
                        cell_bw[cell_str] = [freq, bw]
                        total_bw += bw
                    if items[15] != 'None':
                        freq = int(items[15])
                        cell_str = items[14] + '-' + items[15]
                        bw = 0
                        if freq in bw_freq:
                            bw = bw_freq[freq]
                        elif freq > 2000000:
                            bw = 100
                        cell_bw_grid[gps_str][cell_str] = [lat, lon, freq, bw]
                        cell_bw[cell_str] = [freq, bw]
                        total_bw += bw
                    if items[17] != 'None':
                        freq = int(items[17])
                        cell_str = items[16] + '-' + items[17]
                        bw = 0
                        if freq in bw_freq:
                            bw = bw_freq[freq]
                        elif freq > 2000000:
                            bw = 100
                        cell_bw_grid[gps_str][cell_str] = [lat, lon, freq, bw]
                        cell_bw[cell_str] = [freq, bw]
                        total_bw += bw
                        
                    cell_set_bw_grid[gps_str][cell_set_str] = total_bw
                    
                    cell_set_grid_stat[gps_str][2] += total_bw * ratio
                    cell_set_grid_stat[gps_str][3] = max(cell_set_grid_stat[gps_str][3], total_bw)
                    
                    line_tmp = ""
                    for i in range(0,19):
                        line_tmp += items[i] + ","
                    line_tmp += str(total_bw) + ","
                    for i in range(19,34):
                        line_tmp += items[i] + ","
                        
                    line_cell_set.append(line_tmp)
                
            mw_cell_num_list = []
            sub6_cell_num_list = []
            lte_cell_num_list = []
                
            fout = open(path + "/" + region + "_grid_cell_num.csv", 'w')
            fout.write('grid_lat,grid_lon,cell_num,nr_cell_num,lte_cell_num\n')
            for k1, v1 in cell_bw_grid.items():
                cell_num = 0
                nr_cell_num = 0
                lte_cell_num = 0
                grid_lat = None
                grid_lon = None
                
                mw_cell_num = 0
                sub6_cell_num = 0
                
                for k2, v2 in v1.items():
                    grid_lat = v2[0]
                    grid_lon = v2[1]
                    channel = v2[2]
                    
                    cell_num += 1
                    if channel > 100000:
                        nr_cell_num += 1
                        if channel > 2000000:
                            mw_cell_num += 1
                        else:
                            sub6_cell_num += 1
                    else:
                        lte_cell_num += 1
                if grid_lat != None:
                    if nr_cell_num > 0 or lte_cell_num >= 20:
                        fout.write(str(grid_lat) + ',' + str(grid_lon) + ',' + str(cell_num) + ',' + str(nr_cell_num) + ',' + str(lte_cell_num) + '\n')
                        mw_cell_num_list.append(mw_cell_num)
                        sub6_cell_num_list.append(sub6_cell_num)
                        lte_cell_num_list.append(lte_cell_num)
            fout.close()
            
            fout = open(path + "/" + region + "_mw_cell_num_cdf.csv", 'w')
            mw_cell_num_list.sort()
            length = len(mw_cell_num_list)
            cdf_value = 0
            step = 0
            for i in range(length):
                cdf_value += 1/length
                if cdf_value > step:
                    step += 0.001
                    if mw_cell_num_list[i] == 0:
                        mw_cell_num_list[i] = 0.5
                    fout.write(str(mw_cell_num_list[i]) + ','+ str(cdf_value) + '\n')
            fout.close()
            
            fout = open(path + "/" + region + "_sub6_cell_num_cdf.csv", 'w')
            sub6_cell_num_list.sort()
            length = len(sub6_cell_num_list)
            cdf_value = 0
            step = 0
            for i in range(length):
                cdf_value += 1/length
                if cdf_value > step:
                    step += 0.001
                    if sub6_cell_num_list[i] == 0:
                        sub6_cell_num_list[i] = 0.5
                    fout.write(str(sub6_cell_num_list[i]) + ','+ str(cdf_value) + '\n')
            fout.close()
            
            fout = open(path + "/" + region + "_lte_cell_num_cdf.csv", 'w')
            lte_cell_num_list.sort()
            length = len(lte_cell_num_list)
            cdf_value = 0
            step = 0
            for i in range(length):
                cdf_value += 1/length
                if cdf_value > step:
                    step += 0.001
                    if lte_cell_num_list[i] == 0:
                        lte_cell_num_list[i] = 0.5
                    fout.write(str(lte_cell_num_list[i]) + ','+ str(cdf_value) + '\n')
            fout.close()


FileNotFoundError: [Errno 2] No such file or directory: '/Users/yanbingliu/Dropbox/infocom23-dataset/grid/A-R1_grid_cell_set_pixel5_20210405_20220127_310410_0.0005_0_0.csv'